In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
import nltk
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.text import one_hot
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
import json
import random
import csv
import re
import pickle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f = open('/content/drive/MyDrive/Internship/Internship/tweets-2016-10000-textonly.txt', 'r')
lines = f.readlines()

tweets = []
labels = []

len_train = 1000

In [ ]:
with open('/content/drive/MyDrive/Internship/Internship/IRAhandle_tweets_1.csv', newline='') as csvfile:
    categories = csvfile.readline().split(",")
    tweetreader = csv.reader(csvfile, delimiter=',')
    counter = 0
    for row in tweetreader:
        tweet = dict(zip(categories, row))
        if tweet['language'] == 'English':
            tweets.append(tweet['content'])
            labels.append(1)
            counter += 1
        if counter > len_train:
            break
csvfile.close()

# In[ ]:

for line in lines:
    # for line in lines:
    #     tweet = json.loads(line)
    #     if 'user' in tweet.keys():
    #         if tweet["user"]["lang"] == "en":
    #             tweets.append(tweet['text'])
    #             labels.append(0)
    tweets.append(line)
    labels.append(0)

f.close()
            
tweets_to_labels = dict(zip(tweets, labels))
random.shuffle(tweets)

actual = []

for tweet in tweets:
    actual.append(tweets_to_labels[tweet])
data=pd.DataFrame()
data['Text']=tweets
data['labels']=actual
data

,Text,labels
0,The Trump we've come to know and love As news ...,0
1,Donald Trump pudo haber evitado el pago de imp...,0
2,Kollitja e Hillary Clinton është rishfaqur sër...,0
3,Just like Donald Trump they're not ready to ma...,0
4,"Trump uses tax laws, Hillary breaks laws,pleas...",0
...,...,...
8968,@ABCNews24 \n,0
8969,Voters need to analyze both candidates and wha...,0
8970,Holier then thou @CNN @FWhitfield airs 1988 Fm...,0
8971,"'Mommy,am I gonna die':Mom of 5year-old Ohio g...",1


In [ ]:
tokenizer=ToktokTokenizer()
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
X=data.drop('labels',axis=1)
y=data['labels']
y.value_counts()

0    7972
1    1001
Name: labels, dtype: int64

In [ ]:
tweets=X.copy()
tweets.reset_index(inplace=False)
tweets

,Text
0,The Trump we've come to know and love As news ...
1,Donald Trump pudo haber evitado el pago de imp...
2,Kollitja e Hillary Clinton është rishfaqur sër...
3,Just like Donald Trump they're not ready to ma...
4,"Trump uses tax laws, Hillary breaks laws,pleas..."
...,...
8968,@ABCNews24 \n
8969,Voters need to analyze both candidates and wha...
8970,Holier then thou @CNN @FWhitfield airs 1988 Fm...
8971,"'Mommy,am I gonna die':Mom of 5year-old Ohio g..."


In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(tweets)):
    print(i)
    tweet = re.sub('[^a-zA-Z]', ' ', tweets['Text'][i])
    
    tweet = tweet.lower()
    tweet = tweet.split()
    
    tweet = [ps.stem(word) for word in tweet if not word in stopwords.words('english')]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

Streaming output truncated to the last 5000 lines.
3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162

In [ ]:
corpus

['trump come know love news trump tax break goe script ralli pennsylvania http co jbkrdd sj',
 'donald trump pudo haber evitado el pago de impuesto durant os donald trump candidato republicano l http co irotx qk',
 'kollitja e hillari clinton sht rishfaqur rish ngjesin e premt',
 'like donald trump readi make america great restor former glori dream nightmar past',
 'trump use tax law hillari break law pleas give us respect basic angri third grader http co yulvvggyzr',
 'tecat se burla de trump con comerci dentro del debat con clinton http co kfliahgw http co n r dd x',
 'trump clinton statist tie new poll fl voter http co fqwiu g zu http co fmhstzfgi',
 'import updat spread tweet http co kc de qxdr',
 'caught tape hillari clinton reveal real feel berni sander true believ http co owwrqtcsh',
 'bill clinton que se encontraba en la pista de despegu conversando con el secretario de estado jhon kerri',
 'reliabl sourc say email leak reveal clinton ignor call aid nigerian princ http co vgqwx

In [ ]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(corpus).toarray()
y=data.iloc[:,1].values

In [ ]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("vectorizer.pkl","wb")
pickle.dump(cv, pickle_out)
pickle_out.close()

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train, y_train)

from sklearn import metrics
y_pred=model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9013927576601671


In [ ]:
pip install "scikit-learn==0.24.0"

In [ ]:
# Create a Pickle file using serialization 
import pickle
pickle_out = open("classifier_NB.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [ ]:
# Training model using Logistic Regression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)



LogisticRegression()

In [ ]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("classifier_LR.pkl","wb")
pickle.dump(lr, pickle_out)
pickle_out.close()

In [ ]:
y_pred=lr.predict(X_test)


print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.9075208913649025


In [ ]:
# Training model using Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=1)
rfc.fit(X_train, y_train)


RandomForestClassifier(random_state=1)

In [ ]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("classifier_RF.pkl","wb")
pickle.dump(rfc, pickle_out)
pickle_out.close()

In [ ]:
y_pred = rfc.predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

Accuracy: 0.9342618384401115


array([[1598,    2],
       [ 116,   79]])

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)

Accuracy: 0.934819
Precision: 1.000000
Recall: 0.450704
F1 score: 0.621359


In [ ]:
# Training model using SVM
from sklearn import svm
clf = svm.SVC(kernel='rbf', gamma=2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

In [ ]:
vocab_size=5000
one_hot_repr=[one_hot(words,vocab_size) for words in corpus]
one_hot_repr

[[2345,
  4297,
  4960,
  1831,
  3097,
  2345,
  2945,
  1732,
  1369,
  2621,
  2951,
  3058,
  584,
  904,
  793,
  1763],
 [517,
  2345,
  4971,
  848,
  2578,
  3291,
  2055,
  2941,
  4456,
  1764,
  551,
  517,
  2345,
  1590,
  4804,
  4453,
  584,
  904,
  3416,
  1231],
 [655, 4200, 4574, 2590, 3418, 1906, 3713, 1155, 4200, 1578],
 [770, 517, 2345, 878, 61, 869, 3976, 3749, 3670, 4669, 3841, 1245, 4148],
 [2345,
  3283,
  2945,
  473,
  4574,
  1732,
  473,
  2737,
  4844,
  1212,
  3783,
  2970,
  3658,
  4084,
  1299,
  584,
  904,
  3565],
 [2266,
  4027,
  3973,
  2941,
  2345,
  1255,
  170,
  2705,
  4643,
  3495,
  1255,
  2590,
  584,
  904,
  1093,
  584,
  904,
  1808,
  788,
  2941,
  674],
 [2345,
  2590,
  3354,
  3615,
  1606,
  575,
  1184,
  4690,
  584,
  904,
  4398,
  2787,
  2569,
  584,
  904,
  2095],
 [4540, 2456, 1073, 3069, 584, 904, 3752, 2941, 471],
 [1056,
  1003,
  4574,
  2590,
  2167,
  1223,
  2866,
  2749,
  4456,
  3664,
  800,
  584,
  904,


In [ ]:
sent_length=40
embedded_docs=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,  904,  793, 1763],
       [   0,    0,    0, ...,  904, 3416, 1231],
       [   0,    0,    0, ..., 1155, 4200, 1578],
       ...,
       [   0,    0,    0, ..., 4055, 4979, 3401],
       [   0,    0,    0, ...,  584,  904, 2972],
       [   0,    0,    0, ...,  904, 4275, 2604]], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=80
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 80)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               72400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 472,501
Trainable params: 472,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train1,y_train1,validation_data=(X_test1,y_test1),epochs=10,batch_size=64)

Epoch 1/10
113/113 [==============================] - 10s 16ms/step - loss: 0.4192 - accuracy: 0.8575 - val_loss: 0.2019 - val_accuracy: 0.9270
Epoch 2/10
113/113 [==============================] - 1s 11ms/step - loss: 0.1264 - accuracy: 0.9540 - val_loss: 0.1996 - val_accuracy: 0.9354
Epoch 3/10
113/113 [==============================] - 1s 11ms/step - loss: 0.0497 - accuracy: 0.9850 - val_loss: 0.1965 - val_accuracy: 0.9387
Epoch 4/10
113/113 [==============================] - 1s 11ms/step - loss: 0.0277 - accuracy: 0.9924 - val_loss: 0.2483 - val_accuracy: 0.9376
Epoch 5/10
113/113 [==============================] - 1s 12ms/step - loss: 0.0167 - accuracy: 0.9968 - val_loss: 0.2881 - val_accuracy: 0.9331
Epoch 6/10
113/113 [==============================] - 1s 11ms/step - loss: 0.0088 - accuracy: 0.9986 - val_loss: 0.3123 - val_accuracy: 0.9287
Epoch 7/10
113/113 [==============================] - 1s 10ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.3447 - val_accuracy: 0.937

In [ ]:
y_pred=model.predict_classes(X_test1)
y_pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=int32)

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)

Accuracy: 0.808914
Precision: 0.086207
Recall: 0.075377
F1 score: 0.080429


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test1,y_pred)

array([[1541,   36],
       [  80,  138]])

In [ ]:
#####Bidirectional LSTM
embedding_vector_features=80
model1=Sequential()
model1.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.1))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 80)            400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               144800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 545,001
Trainable params: 545,001
Non-trainable params: 0
_________________________________________________________________
None


Bidirectional LSTM

In [ ]:
### Finally training BIdirectional LSTM
model1.fit(X_train1,y_train1,validation_data=(X_test1,y_test1),epochs=10,batch_size=64)

Epoch 1/10
113/113 [==============================] - 4s 19ms/step - loss: 0.3981 - accuracy: 0.8712 - val_loss: 0.2114 - val_accuracy: 0.9309
Epoch 2/10
113/113 [==============================] - 2s 14ms/step - loss: 0.1167 - accuracy: 0.9568 - val_loss: 0.1699 - val_accuracy: 0.9387
Epoch 3/10
113/113 [==============================] - 1s 12ms/step - loss: 0.0586 - accuracy: 0.9818 - val_loss: 0.2054 - val_accuracy: 0.9409
Epoch 4/10
113/113 [==============================] - 1s 12ms/step - loss: 0.0256 - accuracy: 0.9929 - val_loss: 0.2445 - val_accuracy: 0.9365
Epoch 5/10
113/113 [==============================] - 1s 12ms/step - loss: 0.0144 - accuracy: 0.9975 - val_loss: 0.2758 - val_accuracy: 0.9331
Epoch 6/10
113/113 [==============================] - 1s 12ms/step - loss: 0.0097 - accuracy: 0.9979 - val_loss: 0.3238 - val_accuracy: 0.9382
Epoch 7/10
113/113 [==============================] - 1s 12ms/step - loss: 0.0172 - accuracy: 0.9948 - val_loss: 0.2822 - val_accuracy: 0.9309

In [ ]:
from sklearn.metrics import accuracy_score
y_pred1=model1.predict_classes(X_test1)
y_pred1
accuracy_score(y_test1,y_pred1)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9353760445682451

In [ ]:

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test1, y_pred1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test1, y_pred1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test1, y_pred1)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test1, y_pred1)
print('F1 score: %f' % f1)

Accuracy: 0.935376
Precision: 0.814815
Recall: 0.605505
F1 score: 0.694737


In [ ]:
model1.save("LSTM_TeamA_model.h5")
print("Saved model to disk")


Saved model to disk


TFIDF

In [ ]:
import joblib
joblib.dump(model1, "data_transformer.joblib")

TypeError: ignored

Accuracy: 0.8952646239554317


Accuracy: 0.8885793871866295


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.8919220055710306


Accuracy: 0.8913649025069638
